In [1]:
import pandas as pd
import spacy
import numpy as np
import random
import seaborn as sns

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier as GB
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import IsolationForest as IF
from sklearn.dummy import DummyClassifier as DC
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
from copy import deepcopy, copy

In [5]:
import spacy

In [6]:
from tqdm import tqdm

In [7]:
tqdm.pandas()

In [8]:
oos_dev_set = pd.read_csv("data/dataset/oos_dev_set.csv", index_col=0)

In [9]:
oos_test_set = pd.read_csv("data/dataset/oos_test_set.csv", index_col=0)

In [10]:
target = "explicit"

In [11]:
oos_dev_set.columns

Index(['lyrics', 'artist', 'song', 'album_name', 'popularity', 'danceability',
       'energy', 'key', 'mode', 'valence', 'tempo', 'duration_ms',
       'acousticness', 'liveness', 'loudness', 'speechiness', 'time_signature',
       'explicit', 'nb_genres', 'year'],
      dtype='object')

In [12]:
metadata_cols = ['popularity', 'danceability','energy', 'key', 'mode', 'valence', 'tempo', 'duration_ms',
'acousticness', 'liveness', 'loudness', 'speechiness', 'time_signature', 'nb_genres', 'year']

# Custom Classifiers

In [13]:
class IsolationForestClassifier(ClassifierMixin, BaseEstimator):
    
    def __init__(self, **params):
        self.estimator = IF(**params)

    def fit(self, X, y):

        self.X_ = X
        self.y_ = y
        
        self.estimator.fit(X)
        
        return self
    
    def transform(self, X):
        
        preds = self.estimator.predict(X)

        return np.array(list(map(lambda p: 0 if p==1 else 1, preds)))
    
    def predict(self, X):
        return self.transform(X)
    
    def get_params(self, deep=True):
        return self.estimator.get_params()
    
    def set_params(self, **parameters):
        self.estimator.set_params(**parameters)
        return self

In [14]:
CV_NB = 5
COMBINATION_NB = 50
METRICS = ["f1", "recall", "f1_macro", "recall_macro"]
FOCUS_METRIC = "recall"

# Metadata Only

In [15]:
oos_dev_metadata = oos_dev_set[metadata_cols]
oos_dev_target = oos_dev_set[target]

In [16]:
clfs = {
    "DUMMY": {
        'model': DC(),
        'params': {
            'clf__strategy': ['most_frequent', 'uniform', 'stratified']
        }
    },
    "GB": {
        'model': GB(),
        'params': {
            'scaler__unit_variance': [True, False],
            'clf__loss': ['deviance', 'exponential'],
            'clf__n_estimators': range(25, 201, 25),
            'clf__learning_rate': np.geomspace(0.01, 0.2, 10),
            'clf__max_depth': [3, 4, 5],
            'clf__max_features': ['sqrt', 'log2', None],
            'clf__min_impurity_decrease': [0.00, 0.03, 0.06, 0.09, 0.12]
        }
    },
    "RF": {
        'model': RF(),
        'params': {
            'scaler__unit_variance': [True, False],
            'clf__criterion': ['gini', 'entropy'],
            'clf__n_estimators': range(25, 201, 25),
            'clf__max_depth': range(3, 9, 1),
            'clf__max_features': ['sqrt', 'log2', None],
            'clf__min_impurity_decrease': [0.00, 0.03, 0.06, 0.09, 0.12],
            'clf__class_weight': ['balanced', 'balanced_subsample', None]
        }
    },
    "IF":{
        'model': IsolationForestClassifier(),
        'params': {
            'scaler__unit_variance': [True, False],
            'clf__n_estimators': range(25, 201, 25),
            'clf__contamination': ['auto', *list(np.geomspace(0.01, 0.15, 10))]
        }
    }
}

In [17]:
rcvs = {}

for key in clfs.keys():
    print(key)
    
    np.random.seed(32)
    random.seed(24)
    
    chosen_clf = clfs[key]['model']
    possible_params = clfs[key]['params']
    
    pipe_model = Pipeline(steps=[("scaler", RobustScaler()),
                                ("clf", chosen_clf)]
                )
        
    rcv = RandomizedSearchCV(pipe_model, possible_params,
                           scoring=METRICS,
                           refit=FOCUS_METRIC,
                           n_iter=COMBINATION_NB, cv=CV_NB, n_jobs=-1,
                           return_train_score=True,
                           verbose=1)

    X = oos_dev_metadata.values
    y = oos_dev_target.values
    
    rcv.fit(X=X, y=y)
    
    rcvs[key] = rcv
    
    print("------------------------")

DUMMY
Fitting 5 folds for each of 3 candidates, totalling 15 fits


/home/jpedrocm/miniconda3/envs/tcc/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 3 is smaller than n_iter=50. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


------------------------
GB
Fitting 5 folds for each of 50 candidates, totalling 250 fits
------------------------
RF
Fitting 5 folds for each of 50 candidates, totalling 250 fits
------------------------
IF
Fitting 5 folds for each of 50 candidates, totalling 250 fits
------------------------


In [22]:
oos_test_metadata = oos_test_set[metadata_cols]
oos_test_target = oos_test_set[target]

oos_metadata_metrics = pd.DataFrame(columns=["train_f1", "train_recall", "train_f1_macro", "train_recall_macro",
                                             "test_f1", "test_recall", "test_f1_macro", "test_recall_macro",
                                             "best_cv_score", "refit_time", "best_params"])

for key, best_rcv in tqdm(rcvs.items()):
        
    np.random.seed(32)
    random.seed(24)
    
    y_dev_pred = best_rcv.predict(X=oos_dev_metadata.values)
    train_f1_bin = f1_score(oos_dev_target.values, y_dev_pred)
    train_recall_bin = recall_score(oos_dev_target.values, y_dev_pred)
    train_f1_macro = f1_score(oos_dev_target.values, y_dev_pred, average='macro')
    train_recall_macro = recall_score(oos_dev_target.values, y_dev_pred, average='macro')

    y_test_pred = best_rcv.predict(X=oos_test_metadata.values)
    test_f1_bin = f1_score(oos_test_target.values, y_test_pred)
    test_recall_bin = recall_score(oos_test_target.values, y_test_pred)
    test_f1_macro = f1_score(oos_test_target.values, y_test_pred, average='macro')
    test_recall_macro = recall_score(oos_test_target.values, y_test_pred, average='macro')
    
    oos_metadata_metrics.loc[key] = [train_f1_bin, train_recall_bin, train_f1_macro, train_recall_macro,
                                     test_f1_bin, test_recall_bin, test_f1_macro, test_recall_macro,
                                     best_rcv.best_score_, best_rcv.refit_time_, str(best_rcv.best_params_)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 17.92it/s]


In [23]:
oos_metadata_metrics

,train_f1,train_recall,train_f1_macro,train_recall_macro,test_f1,test_recall,test_f1_macro,test_recall_macro,best_cv_score,refit_time,best_params
DUMMY,0.057629,0.546584,0.360903,0.525106,0.085960,0.5,0.391508,0.523472,0.484280,0.008734,{'clf__strategy': 'uniform'}
GB,0.536170,0.391304,0.763300,0.694679,0.279070,0.2,0.628129,0.594784,0.267803,0.247795,"{'scaler__unit_variance': True, 'clf__n_estima..."
RF,0.265625,0.844720,0.597341,0.858035,0.301676,0.9,0.599734,0.859091,0.763068,0.385020,"{'scaler__unit_variance': True, 'clf__n_estima..."
IF,0.104132,0.391304,0.500021,0.608411,0.167832,0.4,0.536656,0.624739,0.409470,0.077377,"{'scaler__unit_variance': True, 'clf__n_estima..."


In [26]:
oos_metadata_metrics.to_csv('results/metadata_metrics.csv', index=True)

# Lyrics Only

In [20]:
oos_dev_lyrics = oos_dev_set[['lyrics']]
oos_dev_target = oos_dev_set[target]

In [21]:
clfs = {
    "DUMMY": {
        'model': DC(),
        'params': {
            'clf__strategy': ['most_frequent', 'uniform', 'stratified']
        }
    },
    "GB": {
        'model': GB(),
        'params': {
            'txt__vectorizer__lowercase': [True, False],
            'txt__vectorizer__strip_accents': ['ascii', None],
            'txt__vectorizer__ngram_range': [(1,1), (1,2), (2,2)],
            'txt__vectorizer__max_df': np.geomspace(0.8, 1.0, 10),
            'txt__vectorizer__min_df': range(1, 5, 1),
            'txt__vectorizer__max_features': range(100, 1001, 100),
            'txt__vectorizer__binary': [True, False],
            'txt__vectorizer__norm': ['l1', 'l2'],
            'txt__vectorizer__use_idf': [True, False],
            'txt__vectorizer__smooth_idf': [True, False],
            'txt__vectorizer__sublinear_tf': [True, False],
            'clf__loss': ['deviance', 'exponential'],
            'clf__n_estimators': range(25, 201, 25),
            'clf__learning_rate': np.geomspace(0.01, 0.2, 10),
            'clf__max_depth': [3, 4, 5],
            'clf__max_features': ['sqrt', 'log2', None],
            'clf__min_impurity_decrease': [0.00, 0.03, 0.06, 0.09, 0.12]
        }
    },
    "RF": {
        'model': RF(),
        'params': {
            'txt__vectorizer__lowercase': [True, False],
            'txt__vectorizer__strip_accents': ['ascii', None],
            'txt__vectorizer__ngram_range': [(1,1), (1,2), (2,2)],
            'txt__vectorizer__max_df': np.geomspace(0.8, 1.0, 10),
            'txt__vectorizer__min_df': range(1, 5, 1),
            'txt__vectorizer__max_features': range(100, 1001, 100),
            'txt__vectorizer__binary': [True, False],
            'txt__vectorizer__norm': ['l1', 'l2'],
            'txt__vectorizer__use_idf': [True, False],
            'txt__vectorizer__smooth_idf': [True, False],
            'txt__vectorizer__sublinear_tf': [True, False],
            'clf__criterion': ['gini', 'entropy'],
            'clf__n_estimators': range(25, 201, 25),
            'clf__max_depth': range(3, 9, 1),
            'clf__max_features': ['sqrt', 'log2', None],
            'clf__min_impurity_decrease': [0.00, 0.03, 0.06, 0.09, 0.12],
            'clf__class_weight': ['balanced', 'balanced_subsample', None]
        }
    },
    "IF":{
        'model': IsolationForestClassifier(),
        'params': {
            'txt__vectorizer__lowercase': [True, False],
            'txt__vectorizer__strip_accents': ['ascii', None],
            'txt__vectorizer__ngram_range': [(1,1), (1,2), (2,2)],
            'txt__vectorizer__max_df': np.geomspace(0.8, 1.0, 10),
            'txt__vectorizer__min_df': range(1, 5, 1),
            'txt__vectorizer__max_features': range(100, 1001, 100),
            'txt__vectorizer__binary': [True, False],
            'txt__vectorizer__norm': ['l1', 'l2'],
            'txt__vectorizer__use_idf': [True, False],
            'txt__vectorizer__smooth_idf': [True, False],
            'txt__vectorizer__sublinear_tf': [True, False],
            'clf__n_estimators': range(25, 201, 25),
            'clf__contamination': ['auto', *list(np.geomspace(0.01, 0.15, 10))]
        }
    }
}

In [91]:
rcvs = {}

for key in clfs.keys():
    print(key)
    
    np.random.seed(32)
    random.seed(24)
    
    chosen_clf = clfs[key]['model']
    possible_params = clfs[key]['params']
    
    pipe_model = Pipeline(steps=[("txt", ColumnTransformer([("vectorizer", 
                                                             TfidfVectorizer(), 
                                                             0)])),
                                ("clf", chosen_clf)]
                )
        
    rcv = RandomizedSearchCV(pipe_model, possible_params,
                           scoring=METRICS,
                           refit=FOCUS_METRIC,
                           n_iter=COMBINATION_NB, cv=CV_NB, n_jobs=-1,
                           return_train_score=True,
                           verbose=1)

    X = oos_dev_lyrics.values
    y = oos_dev_target.values
    
    rcv.fit(X=X, y=y)
    
    rcvs[key] = rcv
    
    print("------------------------")

DUMMY
Fitting 5 folds for each of 3 candidates, totalling 15 fits
------------------------
GB
Fitting 5 folds for each of 3 candidates, totalling 15 fits
------------------------
RF
Fitting 5 folds for each of 3 candidates, totalling 15 fits
------------------------
IF
Fitting 5 folds for each of 3 candidates, totalling 15 fits
------------------------


In [92]:
oos_test_lyrics = oos_test_set[['lyrics']]
oos_test_target = oos_test_set[target]

oos_lyrics_metrics = pd.DataFrame(columns=["train_f1", "train_recall", "train_f1_macro", "train_recall_macro",
                                             "test_f1", "test_recall", "test_f1_macro", "test_recall_macro",
                                             "best_cv_score", "refit_time", "best_params"])

for key, best_rcv in tqdm(rcvs.items()):
        
    np.random.seed(32)
    random.seed(24)
    
    y_dev_pred = best_rcv.predict(X=oos_dev_lyrics.values)
    train_f1_bin = f1_score(oos_dev_target.values, y_dev_pred)
    train_recall_bin = recall_score(oos_dev_target.values, y_dev_pred)
    train_f1_macro = f1_score(oos_dev_target.values, y_dev_pred, average='macro')
    train_recall_macro = recall_score(oos_dev_target.values, y_dev_pred, average='macro')

    y_test_pred = best_rcv.predict(X=oos_test_lyrics.values)
    test_f1_bin = f1_score(oos_test_target.values, y_test_pred)
    test_recall_bin = recall_score(oos_test_target.values, y_test_pred)
    test_f1_macro = f1_score(oos_test_target.values, y_test_pred, average='macro')
    test_recall_macro = recall_score(oos_test_target.values, y_test_pred, average='macro')
    
    oos_lyrics_metrics.loc[key] = [train_f1_bin, train_recall_bin, train_f1_macro, train_recall_macro,
                                     test_f1_bin, test_recall_bin, test_f1_macro, test_recall_macro,
                                     best_rcv.best_score_, best_rcv.refit_time_, str(best_rcv.best_params_)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.22s/it]


In [93]:
oos_lyrics_metrics

,train_f1,train_recall,train_f1_macro,train_recall_macro,test_f1,test_recall,test_f1_macro,test_recall_macro,best_cv_score,refit_time,best_params
DUMMY,0.057629,0.546584,0.360903,0.525106,0.085960,0.500000,0.391508,0.523472,0.051872,0.695503,{'clf__strategy': 'uniform'}
GB,0.542986,0.372671,0.767065,0.686335,0.058824,0.033333,0.517716,0.514431,0.143824,8.608728,"{'txt__vectorizer__use_idf': False, 'txt__vect..."
RF,0.368231,0.633540,0.668307,0.791023,0.340909,0.500000,0.648385,0.717958,0.365641,0.997802,"{'clf__n_estimators': 175, 'clf__min_impurity_..."
IF,0.135211,0.149068,0.553984,0.559493,0.109091,0.100000,0.536357,0.533607,0.176754,2.634795,"{'clf__n_estimators': 150, 'clf__contamination..."


In [ ]:
oos_lyrics_metrics.to_csv('results/lyrics_metrics.csv', index=True)

# Lyrics and Metadata

In [25]:
oos_dev_meta_lyrics = oos_dev_set[['lyrics', *metadata_cols]]
oos_dev_target = oos_dev_set[target]

In [26]:
clfs = {
    "DUMMY": {
        'model': DC(),
        'params': {
            'clf__strategy': ['most_frequent', 'uniform', 'stratified']
        }
    },
    "GB": {
        'model': GB(),
        'params': {
            'col__scaler__unit_variance': [True, False],
            'col__vectorizer__lowercase': [True, False],
            'col__vectorizer__strip_accents': ['ascii', None],
            'col__vectorizer__ngram_range': [(1,1), (1,2), (2,2)],
            'col__vectorizer__max_df': np.geomspace(0.8, 1.0, 10),
            'col__vectorizer__min_df': range(1, 5, 1),
            'col__vectorizer__max_features': range(100, 1001, 100),
            'col__vectorizer__binary': [True, False],
            'col__vectorizer__norm': ['l1', 'l2'],
            'col__vectorizer__use_idf': [True, False],
            'col__vectorizer__smooth_idf': [True, False],
            'col__vectorizer__sublinear_tf': [True, False],
            'clf__loss': ['deviance', 'exponential'],
            'clf__n_estimators': range(25, 201, 25),
            'clf__learning_rate': np.geomspace(0.01, 0.2, 10),
            'clf__max_depth': [3, 4, 5],
            'clf__max_features': ['sqrt', 'log2', None],
            'clf__min_impurity_decrease': [0.00, 0.03, 0.06, 0.09, 0.12]
        }
    },
    "RF": {
        'model': RF(),
        'params': {
            'col__scaler__unit_variance': [True, False],
            'col__vectorizer__lowercase': [True, False],
            'col__vectorizer__strip_accents': ['ascii', None],
            'col__vectorizer__ngram_range': [(1,1), (1,2), (2,2)],
            'col__vectorizer__max_df': np.geomspace(0.8, 1.0, 10),
            'col__vectorizer__min_df': range(1, 5, 1),
            'col__vectorizer__max_features': range(100, 1001, 100),
            'col__vectorizer__binary': [True, False],
            'col__vectorizer__norm': ['l1', 'l2'],
            'col__vectorizer__use_idf': [True, False],
            'col__vectorizer__smooth_idf': [True, False],
            'col__vectorizer__sublinear_tf': [True, False],
            'clf__criterion': ['gini', 'entropy'],
            'clf__n_estimators': range(25, 201, 25),
            'clf__max_depth': range(3, 9, 1),
            'clf__max_features': ['sqrt', 'log2', None],
            'clf__min_impurity_decrease': [0.00, 0.03, 0.06, 0.09, 0.12],
            'clf__class_weight': ['balanced', 'balanced_subsample', None]
        }
    },
    "IF":{
        'model': IsolationForestClassifier(),
        'params': {
            'col__scaler__unit_variance': [True, False],
            'col__vectorizer__lowercase': [True, False],
            'col__vectorizer__strip_accents': ['ascii', None],
            'col__vectorizer__ngram_range': [(1,1), (1,2), (2,2)],
            'col__vectorizer__max_df': np.geomspace(0.8, 1.0, 10),
            'col__vectorizer__min_df': range(1, 5, 1),
            'col__vectorizer__max_features': range(100, 1001, 100),
            'col__vectorizer__binary': [True, False],
            'col__vectorizer__norm': ['l1', 'l2'],
            'col__vectorizer__use_idf': [True, False],
            'col__vectorizer__smooth_idf': [True, False],
            'col__vectorizer__sublinear_tf': [True, False],
            'clf__n_estimators': range(25, 201, 25),
            'clf__contamination': ['auto', *list(np.geomspace(0.01, 0.15, 10))]
        }
    }
}

In [27]:
rcvs = {}

for key in clfs.keys():
    print(key)
    
    np.random.seed(32)
    random.seed(24)
    
    chosen_clf = clfs[key]['model']
    possible_params = clfs[key]['params']
    
    pipe_model = Pipeline(steps=[("col", ColumnTransformer([("scaler", RobustScaler(), list(range(1, 16))),
                                                            ("vectorizer", TfidfVectorizer(), 0)])),
                                ("clf", chosen_clf)]
                )
        
    rcv = RandomizedSearchCV(pipe_model, possible_params,
                           scoring=METRICS,
                           refit=FOCUS_METRIC,
                           n_iter=COMBINATION_NB, cv=CV_NB, n_jobs=-1,
                           return_train_score=True,
                           verbose=1)

    X = oos_dev_meta_lyrics.values
    y = oos_dev_target.values
    
    rcv.fit(X=X, y=y)
    
    rcvs[key] = rcv
    
    print("------------------------")

DUMMY
Fitting 5 folds for each of 3 candidates, totalling 15 fits


/home/jpedrocm/miniconda3/envs/tcc/lib/python3.8/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


------------------------
GB
Fitting 5 folds for each of 10 candidates, totalling 50 fits
------------------------
RF
Fitting 5 folds for each of 10 candidates, totalling 50 fits
------------------------
IF
Fitting 5 folds for each of 10 candidates, totalling 50 fits
------------------------


In [28]:
oos_test_meta_lyrics = oos_test_set[['lyrics', *metadata_cols]]
oos_test_target = oos_test_set[target]

oos_meta_lyrics_metrics = pd.DataFrame(columns=["train_f1", "train_recall", "train_f1_macro", "train_recall_macro",
                                             "test_f1", "test_recall", "test_f1_macro", "test_recall_macro",
                                             "best_cv_score", "refit_time", "best_params"])

for key, best_rcv in tqdm(rcvs.items()):
        
    np.random.seed(32)
    random.seed(24)
    
    y_dev_pred = best_rcv.predict(X=oos_dev_meta_lyrics.values)
    train_f1_bin = f1_score(oos_dev_target.values, y_dev_pred)
    train_recall_bin = recall_score(oos_dev_target.values, y_dev_pred)
    train_f1_macro = f1_score(oos_dev_target.values, y_dev_pred, average='macro')
    train_recall_macro = recall_score(oos_dev_target.values, y_dev_pred, average='macro')

    y_test_pred = best_rcv.predict(X=oos_test_meta_lyrics.values)
    test_f1_bin = f1_score(oos_test_target.values, y_test_pred)
    test_recall_bin = recall_score(oos_test_target.values, y_test_pred)
    test_f1_macro = f1_score(oos_test_target.values, y_test_pred, average='macro')
    test_recall_macro = recall_score(oos_test_target.values, y_test_pred, average='macro')
    
    oos_meta_lyrics_metrics.loc[key] = [train_f1_bin, train_recall_bin, train_f1_macro, train_recall_macro,
                                     test_f1_bin, test_recall_bin, test_f1_macro, test_recall_macro,
                                     best_rcv.best_score_, best_rcv.refit_time_, str(best_rcv.best_params_)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.30s/it]


In [29]:
oos_meta_lyrics_metrics

,train_f1,train_recall,train_f1_macro,train_recall_macro,test_f1,test_recall,test_f1_macro,test_recall_macro,best_cv_score,refit_time,best_params
DUMMY,0.057629,0.546584,0.360903,0.525106,0.085960,0.500000,0.391508,0.523472,0.050678,0.786750,{'clf__strategy': 'uniform'}
GB,1.000000,1.000000,1.000000,1.000000,0.210526,0.133333,0.594266,0.563686,0.119171,3.725462,"{'col__vectorizer__use_idf': True, 'col__vecto..."
RF,0.264039,0.832298,0.596814,0.852444,0.315152,0.866667,0.611901,0.852111,0.228460,2.804922,"{'col__vectorizer__use_idf': False, 'col__vect..."
IF,0.156716,0.130435,0.568408,0.557608,0.227273,0.166667,0.601118,0.576627,0.180356,1.875922,"{'col__vectorizer__use_idf': True, 'col__vecto..."


In [ ]:
oos_meta_lyrics_metrics.to_csv('results/meta_lyrics_metrics.csv', index=True)